My room bookcase
=============

Important note:
- width: measure for X-axis
- height: measure for Y-axis
- length: measure for Z-axis

# Klimpen bookcase
http://www.ikea.com/it/it/catalog/products/10353765/

![klimpen full size](./images/klimpen_1.png)
![klimpen detail](./images/klimpen_2.png)

We will build the various part of the bookcase, then we will put them together. A second approach will be shown after (PyPlasm $Q(arg)$ function will be used).
Structure:
- back: as suggested is the back side of our bookcase, it will be attached to the shelves, the side block, the top and the bottom.
- shelves
- top
- bottom
Kplimpen is offered as a 50x70x33 double shelves bookcase. For this woorkshoop we will handle a parametrized (in respec of the dimension and color) Klimpen


Note: measures are in meters

In [6]:
from larlib import *

# Bookcase structure
We will build the various part of the bookcase, then we will put them together.
A second approach will be shown after (PyPlasm $Q(x)$, $Q(values)$, $PROD(funcs)$ function will be used).

Structure:
- back: as suggested is the back side of our bookcase, it will be attached to the shelves, the side block, the top and the bottom.
- shelves
- top
- base

Kplimpen is offered as a 50x70x33 double shelves bookcase. For this woorkshoop we will handle a parametrized (in respect of the dimension and color) Klimpen

## Back of the bookcase

In [7]:
def back(width, height):
    """
    back: builds the back of the bookcase
    :param back: height, width in meters
          thickness is fixed
    :return: a CUBOID. The 3d model for the back
    """ 
    return CUBOID([width,height,0.01]) #0.1 is thickness

In [8]:
b = back(0.58, 0.7)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181c861c90> >

# Shelves
We are going to create one and more shelves.
firstInitSpace returns the proper offset from the base for the first shelf from the bottom. This is usefull when building the bookcase where the traslation operation is used. Traslation operation is not "relative" to a point butabsolute.

In [9]:
def shelf(width, height, length):
    """
    shelf: builds a single shelf
    :param width: width of the shelf
    :return: a CUBOID. The 3D model for a shelf
    """
    return CUBOID([width,height,length])


def shelves(nshelves, heightBack, width, height, length):
    """
    shelves builds <nshelves> shelves
    :param nshelves: number of shelves
    :param heightBack: height of the back, the area where shelves are positionable
    :param width: width of the shelf.
    :param height: height of the shelf.
    :param length: length of the shelf. (z-axis)
    :return: nshelves CUBOIDs. nshelves shelves
    """
    space = heightBack/(nshelves+1)-height
    s = STRUCT(NN(nshelves) ([shelf(width, height, length), T(2)(space)]))
    return s

def shelvesInitSpace(nshelves, heightBack, heightShelf):
    """
    :param nshelves: number of shelves 
    :param heightBack: height of the back
    :param heightShelf: height of the shelf
    :return: Long. Space from the first shelf and the base
    """
    space =  heightBack/(nshelves+1)-heightShelf
    return  space

In [11]:
ss = shelves(nshelves=2, heightBack=0.58, width=0.68, height=0.02, length=0.3)
VIEW(ss)
VIEW(T(2)(shelvesInitSpace(2, 0.58, 0.02))(ss))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181c876b70> >

![2 shelves](./images/shelves.png)

## Sides
The lateral components. Two parallel wooden beam

In [13]:
def side(heigth, length):
    """
    side: side of the bookcase structure
    :param heigth: height of the wooden beam
    :param length: length of the wooden beam
    :return: a CUBOID. The 3d model for a lateral wooden beam
    """
    return CUBOID([0.03, heigth, length])

In [14]:
def sides(height, length, distance):
    """
    sides: a couple of sides
    :param height: as side(height,length)
    :param length:  "
    :param distance:  "
    :return: STRUCT([CUBOID,CUBOID])
    """
    s = side(height, length)
    sides = STRUCT([s, T(1)(distance)(s)])
    return sides

In [14]:
single_side = side(0.58, 0.3)
VIEW(single_side)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181e6cd060> >

In [11]:
two_sides = sides(0.58, 0.3, 0.7)
VIEW(two_sides)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181a533060> >

The lateral components of our bookcase

![sides](./images/sides.png)

## Top
The top of our bookcase. It consists of a set of CUBOIDs placed one on top of another

In [15]:
def top(widthSmall, heightSmall, lengthSmall, widthBookcase, lengthBookcase):
    """
    top: the top of the bookcase, cmade of two different layers/block
    _____________________
    [                   ]
    [      layerBig     ]
    [___________________]
      [ layerSmall    ]
      [_______________]
    
    :param widthSmall: the width of layerSmall, the same of the shelf
    :param heightSmall: the height, see above
    :param lengthSmall: the length, see above
    
    :param widthBookcase: the width of the back side of the boockase will be used to calculate the top width
    :param lengthBack: the length of the back
    :return: a STRUCT made of CUBOIDs (layerBig/small).
    """
    #layer small should have the same dimension of the shelves
    layerSmall = shelf(widthSmall, heightSmall, lengthSmall)
    
    #offset between the two layers
    offX = (widthBookcase-widthSmall)/2
    offZ = (lengthBookcase-lengthSmall)/2
    
    layerBig = CUBOID([widthBookcase, 0.03, lengthBookcase])
    
    return STRUCT([T([1,3])([offX,offZ])(layerSmall), T(2)(heightSmall)(layerBig)])

# Usefull variables
This value should be aproximately the ones for the standard ikea Klimpen but may be customized.
Sizes will be used in form of tuples for practical use, so that arguments list size will not be huge in future. 

In [15]:
sizes = {
'backWidth': 0.61,
'backHeight': 0.48,
'shelfWidth': 0.59,
'shelfHeight': 0.02,
'shelfLength': 0.3,
'bookcaseWidth': 0.7,
'bookcaseHeight': 0.5,
'bookcaseLength': 0.33
}

We should keep in mind that the bottom and small layer of the top is the same size as a shelf

In [17]:
t = top(sizes['shelfWidth'], sizes['shelfHeight'], sizes['shelfLength'], sizes['shelfWidth']+0.04, sizes['shelfLength']+0.04)
VIEW(t)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181e6ce030> >

![top](./images/top.png)

## Base
As the top the base is made of layers, we have two different layers: 
- the one on the bottom that is the biggest of the two
- the middle one that is very thin and has a smaller perimeter than the lower one

In [18]:
def base(width, length):
    """
    base of fixed height 0.06
    :param width: width of the bookcase
    :param length: length of the bookcase
    :return: STRUCT. Two CUBOIDs representing the base
    """
    
    layerBig = CUBOID([width, 0.05, length])
    layerSmall = CUBOID([0.96*width, 0.01, 0.96*length])
    
    #offsets. layerSmall has a 0.04^2 smaller perimeter (X,Z)
    offX = width*0.02
    offZ = length*0.02
    
    b = STRUCT([layerBig, T([1, 2, 3])([offX, 0.05, offZ])(layerSmall)])
    return b

In [20]:
b = base(sizes['bookcaseWidth'], sizes['bookcaseLength'])
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181fd7b060> >

![base](./images/base.png)

# Let's build the KLIMPEN bookcase
Now we will create a model for the whole bookcase (Remember to read ikea's instructions).
The ggpl_KLIMPEN function takes as reference top/base width and length.

In [19]:
def ggpl_KLIMPEN(nshelves, sizesDict, colorHEX):
   
    # base
    baseKlimpen = base(sizesDict['bookcaseWidth'], sizesDict['bookcaseLength'])
    
    # back
    # back is traslated according to the position of the base 
    backKlimpen = T([1, 2, 3])([0.03, 0.06, 0.01])(back(sizesDict['backWidth'],sizesDict['backHeight']))
    
    # Top
    # top is placed upon the structure with the bottom layer between the sides,
    # while the upper layer goes above the whole structure
    topOffsetY = 0.06 + sizesDict['backHeight'] - sizesDict['shelfHeight']                          # 0.06 for the base
    topKlimpen = T([2])([topOffsetY])(top(sizesDict['shelfWidth'], sizesDict['shelfHeight'], sizesDict['shelfLength'], sizesDict['bookcaseWidth'], sizesDict['bookcaseLength']))
    
    # sides
    # sidesOffsetX is the offset on the x-axis. 
    sidesOffsetX = (sizesDict['bookcaseWidth'] - sizesDict['backWidth'] + 0.03)/4
    sidesKlimpen = T([1, 2, 3])([sidesOffsetX, 0.06, 0.01])(sides(sizesDict['backHeight'], sizesDict['shelfLength']+0.01, sizesDict['backWidth']))
    
    # shelves
    # shelves are traslated according to the position of the base, the width of the sides and the back
    shelvesOffsetY = shelvesInitSpace(nshelves, sizesDict['backHeight'], sizesDict['shelfHeight']) + 0.06    # base
    shelvesOffsetX = sidesOffsetX                                                                            # sides
    shelvesOffsetZ = 0.02
    shelvesKlimpen = T([1, 2, 3])([shelvesOffsetX, shelvesOffsetY, shelvesOffsetZ])(shelves(nshelves, sizesDict['backHeight'], sizesDict['shelfWidth'], sizesDict['shelfHeight'], sizesDict['shelfLength']))
    
    #klimpen = HEX(colorHEX)(STRUCT([baseKlimpen, backKlimpen, sidesKlimpen, shelvesKlimpen, topKlimpen]))
    klimpen = HEX(colorHEX)(STRUCT([backKlimpen, shelvesKlimpen,sidesKlimpen, topKlimpen, baseKlimpen]))
    return klimpen
    

In [29]:
myKlimpen = ggpl_KLIMPEN(1, sizes, "#C8C5C0")
VIEW(myKlimpen)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10eb7cf00> >

### Klimpen finally rendered

![klimpen1](./images/klimpenRender1.png)

![klimpen2](./images/klimpenRender2.png)

![klimpen3](./images/klimpenRender3.png)

# Let's put Klimpen in a room

In [30]:
def simpleWalls(x, y, colorWallHEX, colorCeilingHEX, colorFloorHex):
    background = CUBOID([x, y, 0.01])
    leftWall = CUBOID([0.01, y,x])
    rightwall = T(1)(x)(leftWall)
    ceiling = HEX(colorCeilingHEX)(T(2)(y)(CUBOID([x,0.01,x])))
    floor = HEX(colorFloorHex)(CUBOID([x,0.01,x]))
    
    walls = HEX(colorWallHEX)(STRUCT([background, leftWall, rightwall]))
    room = STRUCT([walls, ceiling, floor])
    
    return room

In [33]:
myWalls = simpleWalls(2, 1, colorWallHEX="#98e2d2", colorCeilingHEX="#ffffff", colorFloorHex="#89715d")
VIEW(myWalls)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10ebb2690> >

![empty room](./images/emptyroom.png)

In [34]:
def furnishRoom(room, bookcase):
    roomFurnished = STRUCT([room, T([1,2,3])([0.5,0.01,0.02])(bookcase)])     #no meaningfull coordinate for our bookcase
    return roomFurnished

In [35]:
myFurnishedRoom = furnishRoom(myWalls, myKlimpen)
VIEW(myFurnishedRoom)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10ebb2ae0> >

![furnished room](./images/furnished.png)

In [36]:
def book(bookColorHEX):
    coverback = CUBOID([0.01, 0.09, 0.05])
    spine = CUBOID([0.05, 0.09, 0.01])
    
    external = HEX(bookColorHEX)(STRUCT([coverback, spine, T(1)(0.05)(coverback)]))
    pages = HEX("#ffffff")(T([1,2,3])([0.01, 0.01, 0.01])(CUBOID([0.04, 0.075, 0.035])))
    
    return STRUCT([external, pages])

In [37]:
myBook = book("#dd0000")
VIEW(myBook)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181e6cbd50> >

In [40]:
def addBooks(room, bookcase):
    fr = furnishRoom(room, bookcase)
    b1 = T([1,2,3])([0.57, 0.06, 0.07])(book("#dd0000"))
    b2 = T([1,2,3])([0.64, 0.06, 0.06])(book("#00dd00"))
    b3 = T([1,2,3])([0.75, 0.06, 0.09])(R([1, 3])(PI)(book("#0000dd")))
    return STRUCT([fr, b1, b2, b3])

In [41]:
scene = addBooks(myFurnishedRoom, myKlimpen)
VIEW(scene)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181e6cbab0> >

![books](./images/books.png)

# Q, QUOTE and PROD in action
As mentioned at the beginning of this notebook we will now use Pyplasm Q, QUOTE and PRODE operands.
- Q: transform a given digit n in input as a 1D polyhedra of n length
- QUOTE: transforms non-empty sequences of non-zero reals into 1D polyhedra. Positive numbers produce solid segments; negative numbers are used as translations: QUOTE(2,-10,1,1,-10,2)
- PROD: cartesian product of sequences or functions. Q(3)


In [42]:
def topQCart(nDivisors):
    """
    example: topQCart(2)
    :param nDivisors: number of divisors that is the vertical sections
    :return: pyplasm.xgepy.Hpc  . the top
    """
    # 0.2 = difference layerSmall and layerBig, both on left and right, 0.02 fro the first side
    xSpace = 0.02*2+0.03+(0.59+0.03)*nDivisors
    layerBig = INSR(PROD)([Q(0.02*2+0.03+(0.59+0.03)*nDivisors), QUOTE([-0.02, 0.03]), Q(0.34)])

    #layer small should have the same dimension of the shelves
    xRithm = AL([-0.03-0.02,[0.59,-0.03]*nDivisors])           # -0.03-0.02 side offset
    layerSmall = INSR(PROD)([QUOTE(xRithm), Q(0.02), QUOTE([-0.02, 0.3])])

    return STRUCT([layerSmall, layerBig])

In [47]:
def bookcaseBodyQCart(nShelves, nDivisors):
    """
    bookcaseBodyQCart(2,3)
    :param nShelves: number of shelves
    :param nDivisors: number of divisors
    :return: pyplasm.xgepy.Hpc
    """
    xRithm = QUOTE([0.03, -0.59]*(nDivisors+1))
    vertBeams = INSR(PROD)([xRithm, Q(0.22*(nShelves+1)), Q(0.3)])

    if nShelves == 0:
        return STRUCT([vertBeams])
    else:
        #shelves = INSR(PROD)([QUOTE([-0.03, 0.59]*nDivisors), QUOTE(AL[-0.06, [-0.2, 0.02]*nShelves]), Q(0.3)])
        shelves = INSR(PROD)([QUOTE([-0.03, 0.59]*nDivisors), QUOTE([-0.2, 0.02]*nShelves), Q(0.3)])
        return STRUCT([shelves, vertBeams])

In [50]:
VIEW(topQCart(3))
VIEW(bookcaseBodyQCart(2,3))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181e6cba50> >

In [51]:
def baseQCart(nDivisors):
    """
    baseQCart(2,3)
    :param nDivisors: number of divisors
    :return: pyplasm.xgepy.Hpc
    """
    layerBig = INSR(PROD)([Q(0.02*2+0.03+(0.59+0.03)*nDivisors), Q(0.05), Q(0.34)])
    layerSmall = INSR(PROD)([QUOTE([-0.02,(0.03+(0.59+0.03)*nDivisors)]), QUOTE([-0.05, 0.01]), QUOTE([-0.01,0.32])])
    return STRUCT([layerSmall, layerBig])

In [52]:
VIEW(baseQCart(3))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181e6cbf00> >

In [53]:
def ggpl_KLIMPEN_QCart(nShelves, nDivisors):
    """
    bookcaseBodyQCart(2,3)
    :param nShelves: number of shelves
    :param nDivisors: number of divisors
    :return: pyplasm.xgepy.Hpc
    """
    topKlimpen = T(2)(0.22*(nShelves+1)+0.04)(topQCart(nDivisors))
    bodyKlimpen = T([1,2,3])([0.02, 0.06, 0.02])(bookcaseBodyQCart(nShelves, nDivisors))
    baseKlimpen = baseQCart(nDivisors)
    return STRUCT([topKlimpen, bodyKlimpen, baseKlimpen])

In [54]:
VIEW(ggpl_KLIMPEN_QCart(2,3))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181e6cb240> >

![klimpen2](./images/klimpenQCART.png)
![klimpen2](./images/klimpenQCART2.png)